In [1]:
!pip install bpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.4/371.4 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 39.2 MB/s eta 0:00:00


In [2]:
# @title Default title text
from google.colab import drive
import os
import time
import math

drive.mount('/content/drive', force_remount=True)

# dataset:
# fbx files (main dataset source)
FBX = '/content/drive/MyDrive/AutoCalibr/dataset/fbx/'
# ply (pre-converted to fix artifacts)
PLY_TO_IMPORT = '/content/drive/MyDrive/AutoCalibr/dataset/ply/'

# intermediate folders
INTERMEDIATES = '/content/intermediates/'
CONVERTED_PLY = '/content/intermediates/converted_ply/'
NORMALIZED_PLY = '/content/intermediates/normalized_ply/'
# NORMALIZED_PLY = '/content/drive/MyDrive/AutoCalibr/normalized_ply/'


# sometimes used for debug outputting into non-cluttered directory
DIR = '/content/'

# these will be automatically determined, do not change them (defined here to allow out-of-order exports)
global_stretch_scale_x = 1
global_stretch_scale_y = 1
global_stretch_scale_z = 1

global_stretch_offset_x = 0
global_stretch_offset_y = 0
global_stretch_offset_z = 0

Mounted at /content/drive


In [30]:
class Vertex:
  def __init__(self, x, y, z):
    self.x = x
    self.y = y
    self.z = z

  def scale(self, scale_x, scale_y, scale_z):
    self.x *= scale_x
    self.y *= scale_y
    self.z *= scale_z

  def translate(self, offset_x, offset_y, offset_z):
    self.x += offset_x
    self.y += offset_y
    self.z += offset_z

  def __hash__(self):
    return hash((self.x, self.y, self.z))

  def __eq__(self, other):
    if isinstance(other, Vertex):
      return self.x == other.x and self.y == other.y and self.z == other.z
    return False


class Face:
  def __init__(self, vertices):
    # vertices is an unordered list of the indexes of the connected vertices that form the face
    self.vertices = vertices

  def __lt__(self, other):
    for val1, val2 in zip(self.vertices, other.vertices):
      if val1 < val2:
        return True
      elif val1 > val2:
        return False
    return len(self) < len(other)

  def __hash__(self):
    return hash(tuple(self.vertices))

  def __eq__(self, other):
    if isinstance(other, Face):
      return (sorted(self.vertices) == sorted(other.vertices))
    return False


class PlyObject:
  def __init__(self, name, vertices, faces):
    self.name = name
    self.vertices = vertices
    self.faces = faces


  def save_file(self, filename):
    with open(filename, "w") as file:
      file.write("ply\n")
      file.write("format ascii 1.0\n")
      file.write("comment Created by PlyObject class\n")
      file.write(f"element vertex {len(self.vertices)}\n")
      file.write("property float x\n")
      file.write("property float y\n")
      file.write("property float z\n")
      file.write(f"element face {len(self.faces)}\n")
      file.write("property list uchar uint vertex_indices\n")
      file.write("end_header\n")

      for vertex in self.vertices:
        file.write(f"{vertex.x} {vertex.y} {vertex.z}\n")

      for face in self.faces:
        formatted_vertices = ' '.join(str(v) for v in face.vertices)
        number_of_vertices = len(face.vertices)
        file.write(f"{number_of_vertices} {formatted_vertices}\n")


  def scale(self, scale_x, scale_y, scale_z):
    for vertex in self.vertices:
      vertex.scale(scale_x, scale_y, scale_z)


  def translate(self, offset_x, offset_y, offset_z):
    for vertex in self.vertices:
      vertex.translate(offset_x, offset_y, offset_z)


  def remove_duplicate_vertices(self):
    vert_dict = {}
    convert_dict = {}

    new_vertices = []
    # iterate over existing vertices to identify and save unique ones
    for idx, vertex in enumerate(self.vertices):
      if vertex in vert_dict:
        convert_dict[idx] = vert_dict[vertex]
      else:
        # assign a unique index to each vertex
        vert_dict[vertex] = len(new_vertices)
        convert_dict[idx] = len(new_vertices)
        new_vertices.append(vertex)

    # replace original vertices with new, duplicate-free list
    self.vertices = new_vertices

    # apply convert_dict to update face vertices to match new unique indexing
    for face in self.faces:
      face.vertices = [convert_dict[vertex] for vertex in face.vertices]

    # convert faces to set to remove any potential duplicate faces
    self.faces = set(self.faces)


  def get_max_values(self):
    max_values = {'x': None, 'y': None, 'z': None}

    for vertex in self.vertices:
      if max_values['x'] is None or vertex.x > max_values['x']:
        max_values['x'] = vertex.x
      if max_values['y'] is None or vertex.y > max_values['y']:
        max_values['y'] = vertex.y
      if max_values['z'] is None or vertex.z > max_values['z']:
        max_values['z'] = vertex.z

    return max_values


  def get_min_values(self):
    min_values = {'x': None, 'y': None, 'z': None}

    for vertex in self.vertices:
      if min_values['x'] is None or vertex.x < min_values['x']:
        min_values['x'] = vertex.x
      if min_values['y'] is None or vertex.y < min_values['y']:
        min_values['y'] = vertex.y
      if min_values['z'] is None or vertex.z < min_values['z']:
        min_values['z'] = vertex.z

    return min_values


  def normalize_scale(self):
    x_coordinates = [vertex.x for vertex in self.vertices]
    y_coordinates = [vertex.y for vertex in self.vertices]
    z_coordinates = [vertex.z for vertex in self.vertices]

    max_distance = max(x_coordinates + y_coordinates + z_coordinates)
    min_distance = min(x_coordinates + y_coordinates + z_coordinates)
    normalization_range = max_distance - min_distance

    if normalization_range == 0:
      raise ValueError("Normalization range cannot be zero")

    for vertex in self.vertices:
      vertex.x = 2 * (vertex.x - min_distance) / normalization_range - 1
      vertex.y = 2 * (vertex.y - min_distance) / normalization_range - 1
      vertex.z = 2 * (vertex.z - min_distance) / normalization_range - 1


  def squares_to_tris(self):
    new_faces = []
    for face in self.faces:
      if len(face.vertices) == 4:
        # Split the quad into two triangles
        new_faces.append(Face([face.vertices[0], face.vertices[1], face.vertices[2]]))
        new_faces.append(Face([face.vertices[0], face.vertices[2], face.vertices[3]]))
      else:
        # Leave the face as it is
        new_faces.append(face)

    self.faces = new_faces


  def delete_plane(self):
    # Note: only call if the object has a plane artifact!
    max_x_index = max(range(len(self.vertices)), key = lambda index: self.vertices[index].x)
    min_x_index = min(range(len(self.vertices)), key = lambda index: self.vertices[index].x)
    max_z_index = max(range(len(self.vertices)), key = lambda index: self.vertices[index].z)
    min_z_index = min(range(len(self.vertices)), key = lambda index: self.vertices[index].z)

    indices_to_remove = set([max_x_index, min_x_index, max_z_index, min_z_index])

    for f, face in enumerate(self.faces):
      if set(face.vertices).intersection(indices_to_remove) == indices_to_remove:
        del self.faces[f]
        break
    else:
      raise ValueError("Face with all vertices not found")

    for index in sorted(indices_to_remove, reverse = True):
      del self.vertices[index]


  def stretch_to_max(self):
    # note: update scale globals before calling
    self.translate(global_stretch_offset_x, global_stretch_offset_y, global_stretch_offset_z)
    self.scale(global_stretch_scale_x, global_stretch_scale_y, global_stretch_scale_z)


  def revert_stretch(self):
    self.scale(1/global_stretch_scale_x, 1/global_stretch_scale_y, 1/global_stretch_scale_z)
    self.translate(-global_stretch_offset_x, -global_stretch_offset_y, -global_stretch_offset_z)


  def dimension_range(self):
    # Initialize the min and max values with the first vertex coordinates
    xmin = xmax = obj.vertices[0].x
    ymin = ymax = obj.vertices[0].y
    zmin = zmax = obj.vertices[0].z

    # Iterate over all vertices and update the min and max values
    for vertex in obj.vertices:
      xmin = min(xmin, vertex.x)
      xmax = max(xmax, vertex.x)
      ymin = min(ymin, vertex.y)
      ymax = max(ymax, vertex.y)
      zmin = min(zmin, vertex.z)
      zmax = max(zmax, vertex.z)

    xrang = xmax - xmin
    yrang = ymax - ymin
    zrang = zmax - zmin

    return {'x':xrang, 'y':yrang, 'z':zrang}


  def sort_data(self):
    # a map from old vertices to their new indices
    index_map = {}

    # sort the vertices by x first, then y, then z
    sorted_vertices = sorted(
      enumerate(self.vertices),
      key=lambda pair: (pair[1].x, pair[1].y, pair[1].z)
    )
    self.vertices = [pair[1] for pair in sorted_vertices]

    # record the new indices of the vertices in the map
    for i, pair in enumerate(sorted_vertices):
      old_index, _ = pair
      index_map[old_index] = i

    # convert old face lists to new face lists using the index map
    new_faces = []
    for face in self.faces:
      new_face_vertices = [index_map[i] for i in face.vertices]
      new_faces.append(Face(new_face_vertices))

    # replace old face list with new face list, which we first sort

    new_faces.sort()

    self.faces = new_faces


  @classmethod
  def from_file(cls, filepath):
    with open(filepath, 'r') as f:
      if next(f).strip() != "ply":
        raise ValueError("The file being read is not a PLY file.")

      for _ in range(2):
        next(f)

      n_vertices = int(next(f).split()[-1])

      for _ in range(3):
        next(f)

      n_faces = int(next(f).split()[-1])

      for _ in range(2):
        next(f)

      vertices = []
      for _ in range(n_vertices):
        x, y, z = map(float, next(f).split())
        vertices.append(Vertex(x, y, z))

      faces = []
      for _ in range(n_faces):
        face_vertices = list(map(int, next(f).split()[1:]))
        faces.append(Face(face_vertices))

    name = os.path.splitext(os.path.basename(filepath))[0]

    return cls(name, vertices, faces)

In [4]:
#@title { vertical-output: true}
import bpy

!rm -r sample_data/ 2>/dev/null
!mkdir {INTERMEDIATES} 2>/dev/null
!rm -r {CONVERTED_PLY} 2>/dev/null
!mkdir {CONVERTED_PLY}

# needs to be rotated counterclockwise (left) 90 degrees - could be mostly automated (noted below)
needs_extra_rotation = ["Ace Of Spades", "Cantata-57", "Chroma Rush", "Cloudstrike", "Dead Mans Tale", "Duality", "False Promises", "Fugue 55", "Hawkmoon", "Jack Queen King 3", "Mindbenders Ambition", "No Time To Explain", "Ruinous Effigy", "Seven Seraph Carbine", "Seventh Seraph CQC-12", "Seventh Seraph Officer Revolver", "Seventh Seraph SAW", "Seventh Seraph SI-2", "Seventh Seraph VY-7", "The Fourth Horseman", "Trustee", "Witherhoard"]

for f in os.listdir(FBX):
  if f.endswith('.fbx'):
    # Isolate the name of the .fbx file (without extension)
    name = os.path.splitext(os.path.basename(f))[0]

    # Delete all mesh objects to avoid exporting multiple models into the same file
    bpy.ops.object.select_all(action='DESELECT')
    bpy.ops.object.select_by_type(type='MESH')
    bpy.ops.object.delete()

    # Load in FBX file
    bpy.ops.import_scene.fbx(filepath=os.path.join(FBX, f))

    # Select the object
    obj_object = bpy.context.selected_objects[0]
    bpy.context.view_layer.objects.active = obj_object

    # Decide on redirection
    if name in needs_extra_rotation:
      forwards_dir = 'Z'
      # Remove the file from the list as it's been accounted for
      needs_extra_rotation.remove(name)
    else:
      forwards_dir = '-X'

    # Export object to PLY
    bpy.ops.export_mesh.ply(filepath=os.path.join(CONVERTED_PLY, f.replace('.fbx', '.ply')), use_ascii=True, use_mesh_modifiers=True, use_normals=False, use_uv_coords=False, use_colors=False, axis_forward=forwards_dir, axis_up='Y')

    print()

# Ensure that any models that were supposed to receive extra rotation were hit
if len(needs_extra_rotation) > 0:
  print(f"The following models were not hit and may need manual inspection: {needs_extra_rotation}")


FBX version: 7400
Export completed '/content/intermediates/converted_ply/Coldheart.ply' in 0.337

FBX version: 7400
Export completed '/content/intermediates/converted_ply/Vigilance Wing.ply' in 0.206

FBX version: 7400
Export completed '/content/intermediates/converted_ply/Riskrunner.ply' in 0.464

FBX version: 7400
Export completed "/content/intermediates/converted_ply/Skyburner's oath.ply" in 0.247

FBX version: 7400
Export completed '/content/intermediates/converted_ply/Anarchy.ply' in 0.584

FBX version: 7400
Export completed '/content/intermediates/converted_ply/The Last Word.ply' in 0.195

FBX version: 7400
Export completed '/content/intermediates/converted_ply/Lord of Wolves.ply' in 0.153

FBX version: 7400
Export completed '/content/intermediates/converted_ply/Sleeper Simulant.ply' in 0.155

FBX version: 7400
Export completed '/content/intermediates/converted_ply/Huckleberry.ply' in 0.183

FBX version: 7400
Export completed '/content/intermediates/converted_ply/Graviton Lance.p

In [5]:
# use manually converted .ply files as small testing set
!rm -f {CONVERTED_PLY}/*

In [31]:
#@title { vertical-output: true}

ply_objs = []

for f in os.listdir(CONVERTED_PLY):
  if f.endswith('.ply'):
    start_time = time.time()
    ply_obj = PlyObject.from_file(filepath=os.path.join(CONVERTED_PLY, f))
    ply_objs.append(ply_obj)
    elapsed_time = time.time() - start_time

    print(f'Processed into object: {ply_obj.name} (took {elapsed_time:.2f} s)')
    print(f'\nVertice count: {len(ply_obj.vertices)}')
    print(f'Face count: {len(ply_obj.faces)}')
    print('-' * 50)


# Import pre-converted .ply files
for f in os.listdir(PLY_TO_IMPORT):
  if f.endswith('.ply'):
    start_time = time.time()
    ply_obj = PlyObject.from_file(filepath=os.path.join(PLY_TO_IMPORT, f))
    ply_objs.append(ply_obj)
    elapsed_time = time.time() - start_time

    print(f'Processed object: {ply_obj.name} (took {elapsed_time:.2f} s) (manually converted ply)')
    print(f'\nVertice count: {len(ply_obj.vertices)}')
    print(f'Face count: {len(ply_obj.faces)}')
    print('-' * 50)

Processed object: Line in the Sand (took 0.33 s) (manually converted ply)

Vertice count: 24644
Face count: 29692
--------------------------------------------------
Processed object: Rat King (took 0.04 s) (manually converted ply)

Vertice count: 12219
Face count: 24308
--------------------------------------------------
Processed object: Outbreak Perfected (took 0.40 s) (manually converted ply)

Vertice count: 86406
Face count: 28802
--------------------------------------------------
Processed object: Vex Mythoclast (took 0.37 s) (manually converted ply)

Vertice count: 50250
Face count: 16750
--------------------------------------------------
Processed object: Prometheus Lens (took 0.05 s) (manually converted ply)

Vertice count: 13902
Face count: 24149
--------------------------------------------------


In [32]:
#@title { vertical-output: true}

# has a giant rectangular plane originally used as a background, will need to be filtered out
has_plane_artifact = ['Abbadon', 'Blind Perdition', 'Ex Machina', 'Komodo-4FR', 'Nova Mortis', 'Trespasser', 'Vestian Dynasty', 'Vouchsafe']

# Remove plane artifact from manually specified objects

# Could also iterate over all objects and use exceptions,
# but would need to give delete_plane function stricter pre-deletion checking
for obj in ply_objs:
  if obj.name in has_plane_artifact:
    has_plane_artifact.remove(obj.name)

    obj_range = obj.dimension_range()

    # Print the absolute maximum and minimum values
    print(f"Object: {obj.name}")
    print("\nDimension Range:")
    print('X:', obj_range['x'])
    print('Y:', obj_range['y'])
    print('Z:', obj_range['z'])

    start_time = time.time()
    obj.delete_plane()
    elapsed_time = time.time() - start_time

    obj_range = obj.dimension_range()

    # Print the absolute maximum and minimum values
    print(f"\nDimension Range (plane deleted in {elapsed_time:.2f} s):")
    print('X:', obj_range['x'])
    print('Y:', obj_range['y'])
    print('Z:', obj_range['z'])

    print('-' * 50)


# Ensure that any models that were supposed to receive extra rotation were hit
if len(has_plane_artifact) > 0:
  print(f"The following models were not hit and may need manual inspection: {has_plane_artifact}")

The following models were not hit and may need manual inspection: ['Abbadon', 'Blind Perdition', 'Ex Machina', 'Komodo-4FR', 'Nova Mortis', 'Trespasser', 'Vestian Dynasty', 'Vouchsafe']


In [33]:
#@title { vertical-output: true}

face_lengths = {}
for obj in ply_objs:
  count = 0
  for face in obj.faces:
    length = len(face.vertices)

    if length in face_lengths:
      face_lengths[length] += 1
    else:
      face_lengths[length] = 1

    count = count +1

for length, count in face_lengths.items():
  print(f'Vertex count {length}: {count} instances')

Vertex count 3: 122409 instances
Vertex count 4: 1292 instances


In [34]:
#@title { vertical-output: true}

face_lengths = {}
for obj in ply_objs:
  tri_count = 0
  square_count = 0
  for face in obj.faces:
    length = len(face.vertices)

    if length == 3:
      tri_count = tri_count + 1
    elif length == 4:
      square_count = square_count + 1
    else:
      raise Exception(f'Face of size {length}')

  if square_count > 0:
    print(f"Object: {obj.name}")
    print(f"Triangles: {tri_count}")
    print(f"Squares: {square_count}")
    print('-' * 50)

Object: Prometheus Lens
Triangles: 22857
Squares: 1292
--------------------------------------------------


In [35]:
#@title { vertical-output: true}

for obj in ply_objs:
    start_time = time.time()

    obj.squares_to_tris()

    elapsed_time = time.time() - start_time

    print(f'Converted any squares to triangles: {obj.name} (took {elapsed_time:.2f} s)\n')

Converted any squares to triangles: Line in the Sand (took 0.01 s)

Converted any squares to triangles: Rat King (took 0.01 s)

Converted any squares to triangles: Outbreak Perfected (took 0.01 s)

Converted any squares to triangles: Vex Mythoclast (took 0.00 s)

Converted any squares to triangles: Prometheus Lens (took 0.01 s)



In [36]:
#@title { vertical-output: true}

face_lengths = {}
for obj in ply_objs:
  count = 0
  for face in obj.faces:
    length = len(face.vertices)

    if length in face_lengths:
      face_lengths[length] += 1
    else:
      face_lengths[length] = 1

    count = count +1

for length, count in face_lengths.items():
  print(f'Vertex count {length}: {count} instances')

Vertex count 3: 124993 instances


In [37]:
#@title { vertical-output: true}

for obj in ply_objs:
    start_time = time.time()

    obj.remove_duplicate_vertices()

    elapsed_time = time.time() - start_time

    print(f'Removed duplicate vertices and faces: {obj.name} (took {elapsed_time:.2f} s)\n')


Removed duplicate vertices and faces: Line in the Sand (took 0.07 s)

Removed duplicate vertices and faces: Rat King (took 0.03 s)

Removed duplicate vertices and faces: Outbreak Perfected (took 0.10 s)

Removed duplicate vertices and faces: Vex Mythoclast (took 0.06 s)

Removed duplicate vertices and faces: Prometheus Lens (took 0.04 s)



In [38]:
#@title { vertical-output: true}

for obj in ply_objs:
  min_val = float('inf')
  max_val = float('-inf')

  for v in obj.vertices:
    min_val = min(min_val, v.x, v.y, v.z)
    max_val = max(max_val, v.x, v.y, v.z)

  print(f"Object: {obj.name}")
  print("\nAbsolute Extrema in any Dimension:")
  print(f"Overall minimum: {min_val}")
  print(f"Overall maximum: {max_val}")

  obj.normalize_scale()

  min_val = float('inf')
  max_val = float('-inf')

  for v in obj.vertices:
    min_val = min(min_val, v.x, v.y, v.z)
    max_val = max(max_val, v.x, v.y, v.z)

  print("\nAbsolute Extrema in any Dimension (Normalized size):")
  print(f"Overall minimum: {min_val}")
  print(f"Overall maximum: {max_val}")
  print('-' * 50)

Object: Line in the Sand

Absolute Extrema in any Dimension:
Overall minimum: -1.0
Overall maximum: 1.0

Absolute Extrema in any Dimension (Normalized size):
Overall minimum: -1.0
Overall maximum: 1.0
--------------------------------------------------
Object: Rat King

Absolute Extrema in any Dimension:
Overall minimum: -0.31704
Overall maximum: 0.172281

Absolute Extrema in any Dimension (Normalized size):
Overall minimum: -1.0
Overall maximum: 1.0
--------------------------------------------------
Object: Outbreak Perfected

Absolute Extrema in any Dimension:
Overall minimum: -0.619683
Overall maximum: 0.268402

Absolute Extrema in any Dimension (Normalized size):
Overall minimum: -1.0
Overall maximum: 1.0
--------------------------------------------------
Object: Vex Mythoclast

Absolute Extrema in any Dimension:
Overall minimum: -0.912554
Overall maximum: 0.272706

Absolute Extrema in any Dimension (Normalized size):
Overall minimum: -1.0
Overall maximum: 1.0
----------------------

In [39]:
#@title { vertical-output: true}
min_x = min_y = min_z = float('inf')
max_x = max_y = max_z = float('-inf')

for obj in ply_objs:
  for vertex in obj.vertices:
    min_x = min(min_x, vertex.x)
    min_y = min(min_y, vertex.y)
    min_z = min(min_z, vertex.z)
    max_x = max(max_x, vertex.x)
    max_y = max(max_y, vertex.y)
    max_z = max(max_z, vertex.z)

scale_x, scale_y, scale_z = 2/(max_x - min_x), 2/(max_y - min_y), 2/(max_z - min_z)
offset_x, offset_y, offset_z = - (max_x + min_x) / 2, - (max_y + min_y) / 2, - (max_z + min_z) / 2

print(f"Dimensional minimua:")
print(f"X: {min_x}")
print(f"Y: {min_y}")
print(f"Z: {min_z}")

print(f"\nDimensional maxima:")
print(f"X: {max_x}")
print(f"Y: {max_y}")
print(f"Z: {max_z}\n")

print('-' * 50)

print(f"\nDerived Stretch Value:")
print(f"X: {scale_x}")
print(f"Y: {scale_y}")
print(f"Z: {scale_z}")

print(f"\nDerived Offset Value:")
print(f"X: {offset_x}")
print(f"Y: {offset_y}")
print(f"Z: {offset_z}")


Dimensional minimua:
X: -0.07296999999999998
Y: -0.333298
Z: -1.0

Dimensional maxima:
X: 0.6517692320672257
Y: 1.0
Z: 1.0

--------------------------------------------------

Derived Stretch Value:
X: 2.7596132670991973
Y: 1.5000397510534027
Z: 1.0

Derived Offset Value:
X: -0.28939961603361286
Y: -0.333351
Z: -0.0


In [40]:
#@title { vertical-output: true}

# used for checking if I missed any rotation overrides
count_smallest = {'x': 0, 'y': 0, 'z': 0}
count_middle = {'x': 0, 'y': 0, 'z': 0}
count_largest = {'x': 0, 'y': 0, 'z': 0}

# Keep track of absolute min and max for each dimension
track_min = {'x': float('inf'), 'y': float('inf'), 'z': float('inf')}
track_max = {'x': float('-inf'), 'y': float('-inf'), 'z': float('-inf')}

!rm -r {NORMALIZED_PLY} 2>/dev/null
!mkdir {NORMALIZED_PLY}

global_stretch_scale_x = scale_x
global_stretch_scale_y = scale_y
global_stretch_scale_z = scale_z
global_stretch_offset_x = offset_x
global_stretch_offset_y = offset_y
global_stretch_offset_z = offset_z

for obj in ply_objs:
  obj_range = obj.dimension_range()

  print(f"Object: {obj.name}")
  print("\nDimension Range:")
  print('X:', obj_range['x'])
  print('Y:', obj_range['y'])
  print('Z:', obj_range['z'])

  start_time = time.time()
  obj.stretch_to_max()
  elapsed_time = time.time() - start_time

  obj_range = obj.dimension_range()

  # Print the absolute maximum and minimum values
  print(f"\nDimension Range (stretched in {elapsed_time:.2f} s):")
  print('X:', obj_range['x'])
  print('Y:', obj_range['y'])
  print('Z:', obj_range['z'])
  print('-' * 50)

  # counting the podium placings of ranges for every dimension
  sorted_keys = sorted(obj_range, key=obj_range.get)
  smallest_key = sorted_keys[0]
  middle_key = sorted_keys[1]
  largest_key = sorted_keys[2]

  count_smallest[smallest_key] += 1
  count_middle[middle_key] += 1
  count_largest[largest_key] += 1

  # Compare and store the min and max for each dimension

  max_values = obj.get_max_values()
  min_values = obj.get_min_values()

  for dimension in obj_range.keys():
    track_min[dimension] = min(track_min[dimension], min_values[dimension])
    track_max[dimension] = max(track_max[dimension], max_values[dimension])

Object: Line in the Sand

Dimension Range:
X: 0.14593999999999996
Y: 0.671083
Z: 2.0

Dimension Range (stretched in 0.02 s):
X: 0.4027379602004567
Y: 1.0066511762561707
Z: 2.0
--------------------------------------------------
Object: Rat King

Dimension Range:
X: 0.24071723878599127
Y: 1.0167068243545647
Z: 1.6022978780800334

Dimension Range (stretched in 0.01 s):
X: 0.664286485773307
Y: 1.525100651699117
Z: 1.6022978780800334
--------------------------------------------------
Object: Outbreak Perfected

Dimension Range:
X: 0.32267857243394515
Y: 0.7527229938575699
Z: 2.0

Dimension Range (stretched in 0.01 s):
X: 0.8904680694973444
Y: 1.1291144123182812
Z: 2.0
--------------------------------------------------
Object: Vex Mythoclast

Dimension Range:
X: 0.22386311864063568
Y: 0.6348531123972796
Z: 2.0

Dimension Range (stretched in 0.01 s):
X: 0.6177756322148998
Y: 0.9523049046758932
Z: 2.0
--------------------------------------------------
Object: Prometheus Lens

Dimension Range:


In [41]:
#@title { vertical-output: true}

# manually rotated objects so this doesn't matter, but I could use this info to auto-rotate objects if using a large dataset
print("\nTimes dimension had largest dimensional range:")
print(f"X: {count_largest['x']}")
print(f"Y: {count_largest['y']}")
print(f"Z: {count_largest['z']}")

print("\nTimes dimension had middle dimensional range:")
print(f"X: {count_middle['x']}")
print(f"Y: {count_middle['y']}")
print(f"Z: {count_middle['z']}")

print("\nTimes dimension had smallest dimensional range:")
print(f"X: {count_smallest['x']}")
print(f"Y: {count_smallest['y']}")
print(f"Z: {count_smallest['z']}\n")


print('-' * 50)

# shows that our stretch worked
print("\nDimensional Minima:")
print('X:', track_min['x'])
print('Y:', track_min['y'])
print('Z:', track_min['z'])

print("\nDimensional Maxima:")
print('X:', track_max['x'])
print('Y:', track_max['y'])
print('Z:', track_max['z'])


Times dimension had largest dimensional range:
X: 0
Y: 0
Z: 5

Times dimension had middle dimensional range:
X: 0
Y: 5
Z: 0

Times dimension had smallest dimensional range:
X: 5
Y: 0
Z: 0

--------------------------------------------------

Dimensional Minima:
X: -1.0
Y: -0.9999999999999999
Z: -1.0

Dimensional Maxima:
X: 1.0
Y: 0.9999999999999999
Z: 1.0


In [42]:
#@title { vertical-output: true}

for obj in ply_objs:
  start_time = time.time()

  obj.sort_data()

  elapsed_time = time.time() - start_time

  print(f'Sorted object vertices and faces: {obj.name} (took {elapsed_time:.2f} s)\n')

Sorted object vertices and faces: Line in the Sand (took 0.74 s)

Sorted object vertices and faces: Rat King (took 0.18 s)

Sorted object vertices and faces: Outbreak Perfected (took 0.78 s)

Sorted object vertices and faces: Vex Mythoclast (took 0.17 s)

Sorted object vertices and faces: Prometheus Lens (took 0.34 s)



In [43]:
#@title { vertical-output: true}

!rm -r {NORMALIZED_PLY} 2>/dev/null
!mkdir {NORMALIZED_PLY}

for obj in ply_objs:
  start_time = time.time()
  obj.revert_stretch()
  obj.save_file(NORMALIZED_PLY + obj.name + '.ply')
  elapsed_time = time.time() - start_time

  print(f'Exported object: {obj.name} (took {elapsed_time:.2f} seconds)\n')

Exported object: Line in the Sand (took 0.18 seconds)

Exported object: Rat King (took 0.10 seconds)

Exported object: Outbreak Perfected (took 0.08 seconds)

Exported object: Vex Mythoclast (took 0.05 seconds)

Exported object: Prometheus Lens (took 0.08 seconds)



In [20]:
#@title { vertical-output: true}

# Import normalized .ply files (to allow for runtime restart)

ply_objs=[]

for f in os.listdir(NORMALIZED_PLY):
  if f.endswith('.ply'):
    start_time = time.time()
    ply_obj = PlyObject.from_file(filepath=os.path.join(NORMALIZED_PLY, f))
    ply_objs.append(ply_obj)
    elapsed_time = time.time() - start_time

    print(f'Processed object: {ply_obj.name} (took {elapsed_time:.2f} s)')
    print(f'\nVertice count: {len(ply_obj.vertices)}')
    print(f'Face count: {len(ply_obj.faces)}')
    print('-' * 50)

Processed object: Rat King (took 0.12 s) (manually converted ply)

Vertice count: 12219
Face count: 24308
--------------------------------------------------
Processed object: Prometheus Lens (took 0.23 s) (manually converted ply)

Vertice count: 12590
Face count: 25397
--------------------------------------------------
Processed object: Outbreak Perfected (took 0.22 s) (manually converted ply)

Vertice count: 7472
Face count: 28193
--------------------------------------------------
Processed object: Line in the Sand (took 0.25 s) (manually converted ply)

Vertice count: 16496
Face count: 29689
--------------------------------------------------
Processed object: Vex Mythoclast (took 0.03 s) (manually converted ply)

Vertice count: 5198
Face count: 16533
--------------------------------------------------


In [44]:
max_vertices = 0
max_faces = 0
max_total = 0

for obj in ply_objs:
  max_vertices = max(max_vertices, len(obj.vertices))
  max_faces = max(max_faces, len(obj.faces))
  max_total = max(max_total, len(obj.faces)+len(obj.vertices))

print("Maximum vertices: ", max_vertices)
print("Vertex coordinate count (input neurons): ", max_vertices*3)
print("\nMaximum faces: ", max_faces)
print("Each face counts as one input neuron")

Maximum vertices:  16496
Vertex coordinate count (input neurons):  49488

Maximum faces:  29689
Each face counts as one input neuron


Ok I think thats our dataset normalized. AI time.

In [22]:
import ast
import pandas as pd
import warnings
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.utils import pad_sequences

warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

/usr/local/lib/python3.10/dist-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/lib/python3.10/dist-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/usr/local/lib/python3.10/dist-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/lib/python3.10/dist-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [23]:
class Sampling(layers.Layer):
  def call(self, inputs):
    z_mean, z_log_var = inputs
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.random.normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [24]:
latent_dim = 50

Convert our vertex and face data to an array before we define our encoder and decoder so we don't run out of memory.

In [25]:
vertex_inputs_list = []
face_inputs_list = []
for obj in ply_objs:
  vertex_arr = np.array([[vert.x, vert.y, vert.z] for vert in obj.vertices])

  # vertex_arr = pad_sequences(vertex_arr, maxlen=max_vertices, dtype='float32', padding='post')

  vertex_inputs_list.append(vertex_arr)

  print(obj.name)

  obj = None

ply_objs = None

Rat King
Prometheus Lens
Outbreak Perfected
Line in the Sand
Vex Mythoclast


In [26]:
vertex_inputs = keras.Input(shape=(max_vertices, 3))
face_inputs = keras.Input(shape=(max_faces, 1))

x = layers.Dense(32, activation="relu")(vertex_inputs)
x = layers.Flatten()(x)

y = layers.Dense(32, activation="relu")(face_inputs)
y = layers.Flatten()(y)

x = layers.Concatenate()([x, y])

x = layers.Dense(64, activation="relu")(x)
x = layers.Dense(16, activation="relu")(x)

z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])

encoder = keras.Model([vertex_inputs, face_inputs], [z_mean, z_log_var, z], name="encoder")
encoder.summary()


Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 16496, 3)]   0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 29689, 1)]   0           []                               
                                                                                                  
 dense (Dense)                  (None, 16496, 32)    128         ['input_1[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 29689, 32)    64          ['input_2[0][0]']                
                                                                                            

In [27]:
latent_inputs = keras.Input(shape=(latent_dim,))

x = layers.Dense(512, activation="relu")(latent_inputs)
x = layers.Dense(1024, activation="relu")(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(2048, activation="relu")(x)
x = layers.Dense(max_vertices * 3, activation="relu")(x)

vertex_outputs = layers.Reshape((max_vertices, 3))(x)

y = layers.Dense(512, activation="relu")(latent_inputs)
y = layers.Dense(1024, activation="relu")(y)
y = layers.Dropout(0.5)(y)
y = layers.Dense(2048, activation="relu")(y)
y = layers.Dense(max_faces, activation="relu")(y)

face_outputs = layers.Reshape((max_faces, 1))(y)

decoder = keras.Model(latent_inputs, [vertex_outputs, face_outputs], name="decoder")
decoder.summary()

Model: "decoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 50)]         0           []                               
                                                                                                  
 dense_4 (Dense)                (None, 512)          26112       ['input_3[0][0]']                
                                                                                                  
 dense_8 (Dense)                (None, 512)          26112       ['input_3[0][0]']                
                                                                                                  
 dense_5 (Dense)                (None, 1024)         525312      ['dense_4[0][0]']                
                                                                                            

In [28]:
class VAE(keras.Model):
  def __init__(self, encoder, decoder, beta=0.1, **kwargs):
    super().__init__(**kwargs)
    self.encoder = encoder
    self.decoder = decoder
    self.beta = beta
    self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
    self.reconstruction_loss_tracker = keras.metrics.Mean(
      name="reconstruction_loss"
    )
    self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

  @property
  def metrics(self):
    return [
      self.total_loss_tracker,
      self.reconstruction_loss_tracker,
      self.kl_loss_tracker,
    ]

  def train_step(self, data):
    with tf.GradientTape() as tape:
      z_mean, z_log_var, z = self.encoder(data)
      reconstruction = self.decoder(z)
      reconstruction_loss = tf.reduce_mean(
        tf.reduce_sum(
          keras.losses.mse(data, reconstruction), axis=(1, 2)
        )
      )
      kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
      kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
      total_loss = reconstruction_loss + self.beta * kl_loss
    grads = tape.gradient(total_loss, self.trainable_weights)
    self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
    self.total_loss_tracker.update_state(total_loss)
    self.reconstruction_loss_tracker.update_state(reconstruction_loss)
    self.kl_loss_tracker.update_state(kl_loss)
    return {
      "loss": self.total_loss_tracker.result(),
      "reconstruction_loss": self.reconstruction_loss_tracker.result(),
      "kl_loss": self.kl_loss_tracker.result(),
    }

In [29]:
'''
vertex_inputs_arr = np.array(vertex_inputs_list)
face_inputs_arr = np.array(face_inputs_list)

vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit([vertex_inputs_arr, face_inputs_arr], epochs=30, batch_size=128)
'''

'\nvertex_inputs_arr = np.array(vertex_inputs_list)\nface_inputs_arr = np.array(face_inputs_list)\n\nvae = VAE(encoder, decoder)\nvae.compile(optimizer=keras.optimizers.Adam())\nvae.fit([vertex_inputs_arr, face_inputs_arr], epochs=30, batch_size=128)\n'